In [1]:
import pandas as pd
import torch
import numpy as np
from vllm import LLM, SamplingParams
from tqdm.auto import tqdm
import random
import os

2024-11-18 11:59:19.080141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731898759.098048 2169695 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731898759.103752 2169695 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 11:59:19.148853: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/rihankoo/.local/lib/python3.11/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No modu

In [2]:
seed = 9003
# seed = 20241027
# seed = 20240617
# seed = 355643

def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')

set_seed(seed)

> SEEDING DONE


In [3]:
question_nums = 20

In [3]:
MODEL_NAME = "Qwen/Qwen2.5-72B-Instruct-AWQ"
NUM_GPU = torch.cuda.device_count()
few_shot = 3

In [4]:
llm = LLM(
    MODEL_NAME,
    tensor_parallel_size=NUM_GPU,
    gpu_memory_utilization=0.95,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=4096 * 3,
    disable_log_stats=True,
    max_num_seqs=256,
)
tokenizer = llm.get_tokenizer()

sampling_params = SamplingParams(
    n=1,  # Number of output sequences to return for each prompt.
    top_p=0.7,  # Float that controls the cumulative probability of the top tokens to consider.
    temperature=1.0,  # randomness of the sampling
    seed=seed,  # Seed for reprodicibility
    skip_special_tokens=True,  # Whether to skip special tokens in the output.
    max_tokens=4096  # Maximum number of tokens to generate per output sequence.
)

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

INFO 11-14 14:19:05 awq_marlin.py:97] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
WARNING 11-14 14:19:05 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 11-14 14:19:05 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='Qwen/Qwen2.5-72B-Instruct-AWQ', speculative_config=None, tokenizer='Qwen/Qwen2.5-72B-Instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00010-of-00011.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

model-00011-of-00011.safetensors:   0%|          | 0.00/2.49G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/179k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/11 [00:00<?, ?it/s]


INFO 11-14 14:23:28 model_runner.py:1071] Loading model weights took 38.7697 GB
INFO 11-14 14:23:34 gpu_executor.py:122] # GPU blocks: 6846, # CPU blocks: 819
INFO 11-14 14:23:34 gpu_executor.py:126] Maximum concurrency for 8192 tokens per request: 13.37x


In [1]:
import pandas as pd
df = pd.read_csv('../../../../data/raw/misconception_mapping.csv')
df

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...
...,...,...
2582,2582,"When multiplying numbers with the same base, m..."
2583,2583,Does not know what a cube number is
2584,2584,Believes that any percentage of a larger numbe...
2585,2585,Believes a cubic expression should have three ...


In [ ]:
df1 = pd.read_csv('../../../../data/1.chain_of_thought/train.csv')
df1['quality-gpt4o-mini'] = 5
df2 = pd.read_csv('../../../../data/2.synthetic_data_generation/generation2/generation2_all_render.csv')
df3 = pd.read_csv('../../../../data/2.synthetic_data_generation/generation1/generation1_all_render.csv')


In [ ]:
df_base = pd.concat([df1, df2], axis=0).reset_index(drop=True)
df_sup = pd.concat([df3], axis=0).reset_index(drop=True)

In [7]:
len(df_base), len(df_sup)

(82217, 52919)

In [8]:
df_base['MisconceptionId'].value_counts()

MisconceptionId
1214.0    84
2316.0    79
1379.0    77
1880.0    71
1990.0    68
          ..
1391.0     3
1534.0     3
343.0      2
2182.0     1
2224.0     1
Name: count, Length: 2586, dtype: int64

In [9]:
df_sup['MisconceptionId'].value_counts()

MisconceptionId
0       21
1400    21
1540    21
1539    21
1531    21
        ..
1103    17
101     17
2026    17
1905    17
2567    16
Name: count, Length: 2587, dtype: int64

In [10]:
quality_thre = 2
df_base = df_base[df_base['quality-gpt4o-mini'] >= quality_thre].reset_index(drop=True)
df_sup = df_sup[df_sup['quality-gpt4o-mini'] >= quality_thre].reset_index(drop=True)

In [11]:
df_base['MisconceptionId'].value_counts()

MisconceptionId
2316.0    74
1214.0    72
1379.0    69
1507.0    66
1880.0    65
          ..
1391.0     2
2182.0     1
2150.0     1
1001.0     1
928.0      1
Name: count, Length: 2585, dtype: int64

In [12]:
df_sup['MisconceptionId'].value_counts()

MisconceptionId
995     21
81      21
160     21
2121    21
864     21
        ..
2132     3
1080     3
1954     2
277      2
205      2
Name: count, Length: 2587, dtype: int64

In [13]:
ak = df_base['MisconceptionId'].value_counts()
bk = df_sup['MisconceptionId'].value_counts()

ck = {}
for mis_id in df['MisconceptionId']:
    count = 0
    if mis_id in ak:
        count += ak[mis_id]
    if mis_id in bk:
        count += bk[mis_id]
    ck[mis_id] = count

for mis_id in ck:
    if ck[mis_id] < 2:
        print(mis_id, ck[mis_id])

In [14]:
base_misconception_dfs = {}
for mis_id, df_slice in df_base.groupby('MisconceptionId'):
    base_misconception_dfs[mis_id] = df_slice.reset_index(drop=True)

sup_misconception_dfs = {}
for mis_id, df_slice in df_sup.groupby('MisconceptionId'):
    sup_misconception_dfs[mis_id] = df_slice.reset_index(drop=True)

In [15]:
system_prompt_template = '''You are an excellent math teacher about to teach students of year group 1 to 14. You will be provided a misconception that your students may have. Your goal is to write a diagnostic question that can help you identify the misconception. You will be given two math questions, Base Question and Supplementary Question. Please create a new and more challenging question based on the given Questions and following instructions.
To achieve the goal, you have four jobs.
# Add one more step to the Base Question to make it more difficult and Create a new question.
# Changing the proper nouns, numerical values, fractions, multiples and problem background by getting inspiration from the Supplementary Question for generating a new question.
# Check the question step-by-step to find out if it adheres to all principles.
# Modify the created question according to your checking comment to ensure it is of high quality.

You have five principles to do this.
# The question MUST NOT be just asking mathematical concepts, please provide specific context.
# The question must have a single correct answer, please DO NOT ask open questions.
# The question must be useful to identify the misconception, meaning that there is a potential wrong answer relevant to the given misconception.
# Please DO NOT include solution in your question.
# Please generate only one question. The question should be in format of "Subject: <subject>. Topic: <topic>. Subtopic: <subtopic>. Construct: <construct>. Question: <question>"'''

user_prompt_template ='''Misconception: {misconception}
Base Question: Subject: {base_first_subject}. Topic: {base_second_subject}. Subtopic: {base_third_subject}. Construct: {base_construct}. Question:{base_question}
Supplementary Question: Subject: {sup_first_subject}. Topic: {sup_second_subject}. Subtopic: {sup_third_subject}. Construct: {sup_construct}. Question:{sup_question}

Your output should be in the following format:

CREATED QUESTION: Subject: <your created subject>. Topic: <your created topic>. Subtopic: <your created subtopic>. Construct: <your created construct>. Question: <your created question>

VERIFICATION AND MODIFICATION: <check the question step-by-step and modify it to follow all principles>

FINAL CREATED QUESTION: Subject: <your final created subject>. Topic: <your final created topic>. Subtopic: <your final created subtopic>. Construct: <your final created construct>. Question: <your final created question>
'''

In [10]:
i = 40
row = df.iloc[i]
misconception_id = row['MisconceptionId']
misconception = row['MisconceptionName']

p_base = random.random()
p_sup = random.random()
if (misconception_id not in base_misconception_dfs) and (misconception_id in sup_misconception_dfs):
    p_base = 2
    p_sup = -1
elif (misconception_id in base_misconception_dfs) and (misconception_id not in sup_misconception_dfs):
    p_base = -1
    p_sup = 2
elif (len(base_misconception_dfs[misconception_id]) == 1) or (len(sup_misconception_dfs[misconception_id]) == 1):
    if (len(base_misconception_dfs[misconception_id]) == 1) and (len(sup_misconception_dfs[misconception_id]) == 1):
        p_base = -1
        p_sup = -1
    elif len(base_misconception_dfs[misconception_id]) == 1:
        p_sup = -1
    elif len(sup_misconception_dfs[misconception_id]) == 1:
        p_base = -1
if p_base < 0.7 and p_sup < 0.7:
    row_base = base_misconception_dfs[misconception_id].sample(1).iloc[0]
    row_sup = sup_misconception_dfs[misconception_id].sample(1).iloc[0]
elif p_base >= 0.7 and p_sup < 0.7:
    rows = sup_misconception_dfs[misconception_id].sample(2)
    row_base = rows.iloc[0]
    row_sup = rows.iloc[1]
elif p_base < 0.7 and p_sup >= 0.7:
    rows = base_misconception_dfs[misconception_id].sample(2)
    row_base = rows.iloc[0]
    row_sup = rows.iloc[1]
else:
    row_base = sup_misconception_dfs[misconception_id].sample(1).iloc[0]
    row_sup = base_misconception_dfs[misconception_id].sample(1).iloc[0]

base_first_subject = row_base['FirstSubjectName']
base_second_subject = row_base['SecondSubjectName']
base_third_subject = row_base['ThirdSubjectName']
base_construct = row_base['ConstructName']
base_question = row_base['QuestionText']
sup_first_subject = row_sup['FirstSubjectName']
sup_second_subject = row_sup['SecondSubjectName']
sup_third_subject = row_sup['ThirdSubjectName']
sup_construct = row_sup['ConstructName']
sup_question = row_sup['QuestionText']

user_prompt = user_prompt_template.format(base_first_subject=base_first_subject, base_second_subject=base_second_subject, base_third_subject=base_third_subject, base_construct=base_construct, base_question=base_question, sup_first_subject=sup_first_subject, sup_second_subject=sup_second_subject, sup_third_subject=sup_third_subject, sup_construct=sup_construct, sup_question=sup_question, misconception=misconception)
messages = [
    {"role": "system", "content": system_prompt_template},
    {"role": "user", "content": user_prompt},
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(text)

<|im_start|>system
You are an excellent math teacher about to teach students of year group 1 to 14. You will be provided a misconception that your students may have. Your task is to write diagnostic questions that can help you identify the misconception. The question should not be just asking mathematical concepts, so please provide specific context. The question must have a single correct answer, so please do not ask open questions. Only write the question. No need to provide explanation for correct answer. Each question should be in format of "Topic: {subject}. Question: {question}."<|im_end|>
<|im_start|>user
Here are some examples:
example1: 
Misconception: Believes midpoint multiplied by frequency is equivalent to frequency density
Topic: Data Representation-Others. Question: A survey of favorite sports was conducted among 50 students, with the following results: Football (15), Basketball (20), Tennis (10), Swimming (5). If the class width for the sport 'Football' in a histogram i

In [11]:
output = llm.generate(text, sampling_params=sampling_params)
print(output[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:19<00:00, 19.77s/it, est. speed input: 221.44 toks/s, output: 13.00 toks/s]

Topic: Histogram. Question: A histogram shows the distribution of test scores. If 30 students scored between 70-79, and the class width is 10, what is the frequency density for this class?

Topic: Data Representation-Others. Question: A frequency table shows the number of hours students spend on homework each week. If 15 students spent 10-14 hours, and the class width is 5, what is the frequency density for this class?

Topic: Histogram. Question: A histogram represents the ages of participants in a survey. If 45 participants are between 20-29 years old, and the class width is 10, what is the frequency density for this class?

Topic: Data Representation-Others. Question: A frequency table shows the number of books read by students in a year. If 25 students read 5-9 books, and the class width is 5, what is the frequency density for this class?

Topic: Histogram. Question: A histogram displays the heights of a group of trees. If 20 trees are between 15-20 meters tall, and the class width

In [17]:
from tqdm.auto import tqdm
all_texts = []
misconception_ids = []
misconceptions = []
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    misconception_id = row['MisconceptionId']
    misconception = row['MisconceptionName']
    for _ in range(question_nums):
        p_base = random.random()
        p_sup = random.random()
        if (misconception_id not in base_misconception_dfs) and (misconception_id in sup_misconception_dfs):
            p_base = 2
            p_sup = -1
        elif (misconception_id in base_misconception_dfs) and (misconception_id not in sup_misconception_dfs):
            p_base = -1
            p_sup = 2
        elif (len(base_misconception_dfs[misconception_id]) == 1) or (len(sup_misconception_dfs[misconception_id]) == 1):
            if (len(base_misconception_dfs[misconception_id]) == 1) and (len(sup_misconception_dfs[misconception_id]) == 1):
                p_base = -1
                p_sup = -1
            elif len(base_misconception_dfs[misconception_id]) == 1:
                p_sup = -1
            elif len(sup_misconception_dfs[misconception_id]) == 1:
                p_base = -1
        if p_base < 0.7 and p_sup < 0.7:
            row_base = base_misconception_dfs[misconception_id].sample(1).iloc[0]
            row_sup = sup_misconception_dfs[misconception_id].sample(1).iloc[0]
        elif p_base >= 0.7 and p_sup < 0.7:
            rows = sup_misconception_dfs[misconception_id].sample(2)
            row_base = rows.iloc[0]
            row_sup = rows.iloc[1]
        elif p_base < 0.7 and p_sup >= 0.7:
            rows = base_misconception_dfs[misconception_id].sample(2)
            row_base = rows.iloc[0]
            row_sup = rows.iloc[1]
        else:
            row_base = sup_misconception_dfs[misconception_id].sample(1).iloc[0]
            row_sup = base_misconception_dfs[misconception_id].sample(1).iloc[0]

        base_first_subject = row_base['FirstSubjectName']
        base_second_subject = row_base['SecondSubjectName']
        base_third_subject = row_base['ThirdSubjectName']
        base_construct = row_base['ConstructName']
        base_question = row_base['QuestionText']
        sup_first_subject = row_sup['FirstSubjectName']
        sup_second_subject = row_sup['SecondSubjectName']
        sup_third_subject = row_sup['ThirdSubjectName']
        sup_construct = row_sup['ConstructName']
        sup_question = row_sup['QuestionText']

        user_prompt = user_prompt_template.format(base_first_subject=base_first_subject, base_second_subject=base_second_subject, base_third_subject=base_third_subject, base_construct=base_construct, base_question=base_question, sup_first_subject=sup_first_subject, sup_second_subject=sup_second_subject, sup_third_subject=sup_third_subject, sup_construct=sup_construct, sup_question=sup_question, misconception=misconception)
        messages = [
            {"role": "system", "content": system_prompt_template},
            {"role": "user", "content": user_prompt},
        ]

        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        all_texts.append(text)
        misconception_ids.append(misconception_id)
        misconceptions.append(misconception)

  0%|          | 0/2587 [00:00<?, ?it/s]

In [ ]:
output = llm.generate(all_texts, sampling_params=sampling_params)

In [18]:
df_output = pd.DataFrame({'MisconceptionId': misconception_ids, 'MisconceptionName': misconceptions})
df_output

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,0,Does not know that angles in a triangle sum to...
2,0,Does not know that angles in a triangle sum to...
3,0,Does not know that angles in a triangle sum to...
4,0,Does not know that angles in a triangle sum to...
...,...,...
51735,2586,Misunderstands order of operations in algebrai...
51736,2586,Misunderstands order of operations in algebrai...
51737,2586,Misunderstands order of operations in algebrai...
51738,2586,Misunderstands order of operations in algebrai...


In [ ]:
df['synthetic_question'] = [x.outputs[0].text for x in output]

In [ ]:
def extract_question(text):
    topics = []
    questions = []
    splits = text.split('Topic:')
    for split in splits:
        try:
            topic, question = split.split('Question:')
            topic = topic.strip()
            if topic[-1] == '.':
                topic = topic[:-1]
            question = question.strip()
            if question[-1] == '.':
                question = question[:-1]
            topics.append(topic)
            questions.append(question)
        except:
            continue
    return topics, questions

In [ ]:
rows = []
for i in range(len(df)):
    row = df.iloc[i]
    mis_id = row['MisconceptionId']
    mis_name = row['MisconceptionName']
    synthetic_question = row['synthetic_question']
    topics, questions = extract_question(synthetic_question)
    for topic, question in zip(topics, questions):
        rows.append([mis_id, mis_name, topic, question])
df_generated = pd.DataFrame(rows, columns=['MisconceptionId', 'MisconceptionName', 'SubjectName', 'QuestionText'])

In [ ]:
df_generated.to_csv('../../../../data/2.synthetic_data_generation/generation3/generation3_qwen.csv', index=False)